<a href="https://colab.research.google.com/github/mehrnazeraeisi/Diabetes/blob/main/DiabetesMedical_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# Load data
data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop('target', axis=1)
y_continuous = df['target']

# بخش A: خوشه‌بندی با KMeans
def classify_by_kmeans(y, k=3):
    kmeans = KMeans(n_clusters=k, random_state=None)
    y_clusters = kmeans.fit_predict(y.values.reshape(-1, 1))
    return y_clusters

# بخش B: دسته‌بندی پزشکی
def classify_by_rules(y, timing='before'):
    def classify(target):
        if target < 70:
            return 0  # Low
        elif timing == 'before':
            return 1 if target <= 100 else 2  # Normal / High
        elif timing == 'after':
            return 1 if target < 140 else 2
        else:
            return -1
    return y.apply(classify)

# آماده‌سازی داده‌ها
X_train, X_test, y_train_cont, y_test_cont = train_test_split(X, y_continuous, test_size=0.3, shuffle=True)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### مدل A: KMeans + KNN
y_train_kmeans = classify_by_kmeans(y_train_cont)
y_test_kmeans = classify_by_kmeans(y_test_cont)  # فقط برای مقایسه دقیق

knn_kmeans = KNeighborsClassifier(n_neighbors=5)
knn_kmeans.fit(X_train_scaled, y_train_kmeans)
y_pred_kmeans = knn_kmeans.predict(X_test_scaled)

print("🎯 مدل A: بر اساس خوشه‌بندی KMeans")
print("Accuracy:", accuracy_score(y_test_kmeans, y_pred_kmeans))


### مدل B: دسته‌بندی دستی پزشکی + KNN
y_train_manual = classify_by_rules(y_train_cont, timing='before')
y_test_manual = classify_by_rules(y_test_cont, timing='before')

knn_manual = KNeighborsClassifier(n_neighbors=5)
knn_manual.fit(X_train_scaled, y_train_manual)
y_pred_manual = knn_manual.predict(X_test_scaled)

print("\n🩺 مدل B: بر اساس دسته‌بندی پزشکی")
print("Accuracy:", accuracy_score(y_test_manual, y_pred_manual))

🎯 مدل A: بر اساس خوشه‌بندی KMeans
Accuracy: 0.2631578947368421

🩺 مدل B: بر اساس دسته‌بندی پزشکی
Accuracy: 0.6466165413533834
